In [11]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

OSError: Unable to communicate with found ChipWhisperer. Check that 
another process isn't connected to it and that you have permission to communicate with it.

In [77]:
# change this to match the path of your HEX file
HEX_PATH =  "./jsb9786.hex"

In [78]:
# program the board
cw.program_target(scope, cw.programmers.STM32FProgrammer, HEX_PATH)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 23451 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 23451 bytes


In [79]:
## UPDATE THESE KEYS TO MATCH YOUR SETUP

PROTOCOL_NOP = 0x10
PROTOCOL_XOR = 0x20
PROTOCOL_DES = 0x30
PROTOCOL_AES = 0x40

MAGIC_BYTE = 0xFE

RESPONSE_SUCCESS = 0x00
RESPONSE_FAILURE = 0x01
RESPONSE_UNKNOWN_PROTOCOL = 0x02

def response_code_to_string(code):
    code = ord(code)
    if code == RESPONSE_SUCCESS:
        return "SUCCESS"
    elif code == RESPONSE_FAILURE:
        return "FAILURE"
    elif code == RESPONSE_UNKNOWN_PROTOCOL:
        return "UNKNOWN_PROTOCOL"
    else:
        return "UNKNOWN_RESPONSE_CODE"

# reads n bytes from target uart
def read_uart(target, n_bytes):
    response = []
    while len(response) < n_bytes:
        response += target.read()
    return response

In [80]:
target = cw.target(scope, noflush=True) # MUST HAVE noflush=True
print(target.baud) # make sure this baud rate is set in your project

38400


In [81]:
# #convdert the message to a list of bytes
from time import sleep
from Crypto.Cipher import DES
from Crypto.Cipher import AES
message = "I am the senate"

message = [ord(c) for c in message]
#put the magic byte at the beginning of the message
message_plaintext = [MAGIC_BYTE] + message

sleep(0.1)

print("===============MESSAGE_NOP=================")

#make a copy of the message to send with the nop protocol
message_nop = message_plaintext.copy()

#no operation on the message

#prepend the protocol byte
message_nop = [PROTOCOL_NOP] + message_nop

#print the hex representation of the message_nop
print("Message_nop :" + " ".join("{:02x}".format(c) for c in message_nop))
#print(bytes(message_nop))
#write the message to the uart
target.write(bytes(message_nop))

#read the 17 byte response
response_nop = read_uart(target, 17)

print("Response_nop:" + " ".join("{:02x}".format(ord(c)) for c in response_nop))

#check the response code
print("Response code: {}".format(response_code_to_string(response_nop[0])))

#convert the response_nop to a printable string (excluding the response code and magic byte)
response_nop = "".join(response_nop[2:])
print("Response_nop: {}".format(response_nop))

sleep(0.1)

print("===============MESSAGE_XOR=================")

#make a copy of the message to send with the xor protocol
#message_xor = message_plaintext.copy()
message_xor = [0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]

#prepend the protocol byte
message_xor = [PROTOCOL_XOR] + message_xor
print(message_xor)

#print the hex representation of the message_xor
print("Message_xor :" + " ".join("{:02x}".format(c) for c in message_xor))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_xor))

#read the 17 byte response
response_xor = read_uart(target, 17)
XOR_KEY = response_xor.copy()

#print the hex representation of the response
print("Response_xor:" + " ".join("{:02x}".format(ord(c)) for c in response_xor))

#check the response code
print("Response code: {}".format(response_code_to_string(response_xor[0])))

#convert the response_xor to a printable string (excluding the response code and magic byte)
response_xor = "".join(response_xor[2:])
print("Response_xor: {}".format(response_xor))

sleep(0.1)


===============MESSAGE_NOP=================
Message_nop :10 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response_nop:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_nop: I am the senate
===============MESSAGE_XOR=================
[32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Message_xor :20 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Response_xor:01 4a c3 f7 8d 2b 91 5e a2 77 19 bd 6f e8 34 d0 af
Response code: FAILURE
Response_xor: Ã÷+^¢w½oè4Ð¯


In [82]:
#XOR_KEY = [
#    0xa6, 0x81, 0x75, 0x54, 0x13, 0xb9, 0xbc, 0x45, 0x1c, 0x06, 0x8f, 0x99, 0x5f, 0xfb, 0x59, 0x98
#]
XOR_KEY = [ord(c) for c in XOR_KEY]
XOR_KEY = XOR_KEY[1:]
print("XOR_KEY: " + " ".join("{:02x}".format(c) for c in XOR_KEY))

print("===============MESSAGE_XOR=================")

#make a copy of the message to send with the xor protocol
message_xor = message_plaintext.copy()

#xor the message with the key
for i in range(len(message_xor)):
    message_xor[i] ^= XOR_KEY[i]

#prepend the protocol byte
message_xor = [PROTOCOL_XOR] + message_xor

#print the hex representation of the message_xor
print("Message_xor :" + " ".join("{:02x}".format(c) for c in message_xor))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_xor))

#read the 17 byte response
response_xor = read_uart(target, 17)

#print the hex representation of the response
print("Response_xor:" + " ".join("{:02x}".format(ord(c)) for c in response_xor))

#check the response code
print("Response code: {}".format(response_code_to_string(response_xor[0])))

#convert the response_xor to a printable string (excluding the response code and magic byte)
response_xor = "".join(response_xor[2:])
print("Response_xor: {}".format(response_xor))

sleep(0.1)

XOR_KEY: 4a c3 f7 8d 2b 91 5e a2 77 19 bd 6f e8 34 d0 af
===============MESSAGE_XOR=================
Message_xor :20 b4 8a d7 ec 46 b1 2a ca 12 39 ce 0a 86 55 a4 ca
Response_xor:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_xor: I am the senate
